In [2]:
pip install colorgram.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import os
import re
import cv2
import pandas as pd
import colorgram
import numpy as np
from sklearn.cluster import KMeans
from skimage import io
from skimage.color import rgb2lab, deltaE_cie76

# 이미지 파일에서 채도 정보 추출하는 함수
def get_saturation(image_path):
    # 이미지 로드
    image = io.imread(image_path)
    # RGB 값을 LAB 색상 체계로 변환
    lab_image = rgb2lab(image)
    # LAB 체계에서 a, b 축의 값 추출
    a, b = lab_image[:,:,1], lab_image[:,:,2]
    # a, b 축의 값의 표준편차를 이용하여 채도 계산
    saturation = np.sqrt(np.square(a) + np.square(b)).mean()
    return saturation

# 이미지 파일 리스트를 채도 기준으로 정렬하는 함수
def sort_images_by_saturation(image_list):
    # 이미지 파일 리스트와 각 이미지 파일의 채도 값을 저장할 리스트
    saturation_list = []
    sorted_image_list = []

    # 이미지 파일 별로 채도 값 추출
    for image in image_list:
        saturation = get_saturation(os.path.join(folder_path, image))
        saturation_list.append(saturation)

    # 채도 값을 기준으로 이미지 파일 리스트를 정렬
    sorted_index = np.argsort(saturation_list)[::-1]
    for index in sorted_index:
        sorted_image_list.append(image_list[index])

    return sorted_image_list

# 이미지 파일에서 색상 정보 추출하는 함수
def get_colors(image_path):
    # 이미지에서 10개의 색상 추출
    colors = colorgram.extract(image_path, 10)
    # 추출된 색상 정보를 RGB 값으로 변환하여 리스트로 저장
    color_list = [(color.rgb.r, color.rgb.g, color.rgb.b) for color in colors]
    return color_list

# 이미지 파일 리스트를 색상 기준으로 클러스터링하는 함수
def cluster_images_by_color(image_list):
    # 이미지 파일 이름과 색상 정보 저장할 리스트
    name_list = []
    color_list = []

    # 이미지 파일 별로 색상 분석 및 정보 저장
    for image in image_list:
        # 이미지 파일 이름 저장
        name_list.append(image)
        # 색상 분석 및 정보 저장
        color = get_colors(os.path.join(folder_path, image))
        color_array = np.array(color)
        kmeans = KMeans(n_clusters=3, random_state=42).fit(color_array)
        center_colors = kmeans.cluster_centers_
        hex_colors = ['#%02x%02x%02x' % tuple(map(int, color)) for color in center_colors]
        color_list.append(hex_colors)

    # 결과를 데이터프레임으로 저장
    df = pd.DataFrame(color_list, index=name_list, columns=['Color1', 'Color2', 'Color3'])

    return df

# 이미지 파일이 있는 폴더 경로
folder_path = '/content/drive/MyDrive/image/Top'

# 이미지 파일 리스트
image_list = os.listdir(folder_path)

# 이미지 파일 이름과 채도 정보 저장할 리스트
name_list = []
saturation_list = []

# 이미지 파일 별로 채도 분석 및 정보 저장
for image in image_list:
    # 이미지 파일 이름 저장
    name_list.append(image)
    # 이미지 파일 읽어오기
    img = cv2.imread(os.path.join(folder_path, image))
    # 이미지 채도 변환
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    saturation = hsv[:, :, 1].mean()
    # 채도 정보 저장
    saturation_list.append(saturation)

# 결과를 데이터프레임으로 저장
df = pd.DataFrame({'Name': name_list, 'Saturation': saturation_list})

# 채도에 따라 이미지를 클러스터링하는 함수
def cluster_images(df, num_clusters):
    # 채도를 기준으로 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(df[['Saturation']])
    # 클러스터링 결과를 데이터프레임에 추가
    df['Cluster'] = kmeans.labels_
    # 각 클러스터의 대표 이미지를 찾기 위해 채도 값이 가장 큰 이미지를 선택
    df = df.sort_values(by=['Cluster', 'Saturation'], ascending=[True, False])
    df = df.groupby('Cluster').head(1)
    df = df.sort_values(by='Cluster')
    return df

# 클러스터링 결과 출력
df_result = cluster_images(df,4)
print(df_result)

for i in range(4):
  print("cluster:", i)
  cluster_num = i
  cluster_df = df[df['Cluster'] == cluster_num]
  for name in cluster_df['Name']:
     print(name)
  print(" ")


          Name  Saturation  Cluster
69  t (70).jpg   14.089783        0
3    t (5).jpg  139.546875        1
38  t (39).jpg   62.766174        2
58  t (60).jpg   33.061829        3
cluster: 0
t (6).jpg
t (15).jpg
t (17).jpg
t (11).jpg
t (10).jpg
t (16).jpg
t (19).jpg
t (22).jpg
t (25).jpg
t (26).jpg
t (28).jpg
t (23).jpg
t (33).jpg
t (36).jpg
t (41).jpg
t (38).jpg
t (45).jpg
t (44).jpg
t (43).jpg
t (46).jpg
t (50).jpg
t (49).jpg
t (51).jpg
t (52).jpg
t (47).jpg
t (53).jpg
t (55).jpg
t (56).jpg
t (59).jpg
t (61).jpg
t (62).jpg
t (63).jpg
t (64).jpg
t (65).jpg
t (66).jpg
t (67).jpg
t (70).jpg
t (72).jpg
t (74).jpg
t (75).jpg
t (76).jpg
t (77).jpg
t (79).jpg
t (80).jpg
t (83).jpg
t (82).jpg
t (85).jpg
t (86).jpg
t (87).jpg
t (90).jpg
t (92).jpg
t (91).jpg
t (89).jpg
t (96).jpg
t (93).jpg
t (97).jpg
t (98).jpg
 
cluster: 1
t (2).jpg
t (3).jpg
t (5).jpg
t (8).jpg
t (13).jpg
t (21).jpg
t (18).jpg
t (9).jpg
t (68).jpg
t (69).jpg
t (71).jpg
 
cluster: 2
t (4).jpg
t (7).jpg
t (14).jpg
t (35).jpg

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
